In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle
import seaborn as sns
from matplotlib import pyplot as plt
import onnxruntime as rt

In [ ]:
# Load the test data and scaler
with open('artifact/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)
with open('artifact/test_data.pkl', 'rb') as handle:
    (X_test, y_test) = pickle.load(handle)

In [ ]:
# Create a onnx inference runtime session and predict values for all test inputs
sess = rt.InferenceSession("artifact/mode.onnx", providers=rt.get_available_providers())
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
y_pred_temp = sess.run([output_name], {input_name: scaler.transform(X_test).astype(np.float32)}) 
y_pred_temp = np.asarray(np.squeeze(y_pred_temp[0]))
threshold = 0.995
y_pred = np.where(y_pred_temp > threshold, 1,0)

In [ ]:
# Show results
accuracy = np.sum(np.asarray(y_test) == y_pred) / len(y_pred)
print("Accuracy: " + str(accuracy))

c_matrix = confusion_matrix(y_test,y_pred)
ax = sns.heatmap(c_matrix, annot=True, cbar=False, cmap='Blues')
ax.set_xlabel("Prediction")
ax.set_ylabel("Actual")
ax.set_title('Confusion Matrix')
plt.show()

In [ ]:
# Trying with Sally's details
# Fields are in order: distance_from_last_transaction, ratio_to_median_price, used_chip, used_pin_number, online_order 
sally_transaction_details = [[0.3111400080477545, 1.9459399775518593, 1.0, 0.0, 0.0]]
prediction = sess.run([output_name], {input_name: scaler.transform(sally_transaction_details).astype(np.float32)})

print("Was Sally's transaction predicted to be fraudulent?  ")
print(np.squeeze(prediction)>threshold)